In [ ]:
"""New main function with the axe thing"""

#Inventory class
class Inventory:
    def __init__(self):
        self.items = [o10] #will be a list of item objects

    def show_items(self):
        print("In your inventory, you have:\n")
        for item in self.items:
            print(item.name) #for testing purposes only
            #print(item.name) #for actual game

    def add_item(self, thing):
        if thing.weight == 'heavy':
            print("Despite, your best efforts, you cannot manage to lift it")
        else:
            self.items.append(thing)

    def remove_item(self, thing):
        self.items.remove(thing)


#Item class
class Item:
    def __init__(self, name, q_description, s_description, quest, weight):
        self.name = name
        self.q_description = q_description
        self.s_description = s_description
        self.quest = quest
        self.weight = weight
        
    def inspect(self, current_quest):
        if current_quest == self.quest: #if this object is relevant to the quest
            print(self.q_description)
        else:
            print(f"You see a {self.name}.")

#Item objects
o1 = Item('Branch', 'Quest description', 'Standard description', 'Final quest', 'light')
o2 = Item('Human food', "Quest description", 'Standard description', 'Human1', 'light')
o3 = Item('Elven plants', 'Quest description', 'Standard description', 'Elf1', 'light')
o4 = Item('Rose', 'Quest description', 'Standard description', 'Elf2', 'light')
o5 = Item('Crate', 'Quest description', 'Standard description', 'Elf2', 'heavy')
o6 = Item('Bugs', 'Quest description', 'Standard description', 'Elf1', 'light')
o7 = Item('Shovel', 'Quest description', 'Standard description', 'Elf1', 'light')
o8 = Item('Antidote', 'Quest description', 'Standard description', 'Elf1', 'light')
o9 = Item('Bug trap', 'Quest description', 'Standard description', 'Elf1', 'light')
o10 = Item('Axe', 'Quest description', 'Standard description', 'NaN', 'light')

#Location class
class Location:
    def __init__(self, name, message, items, npcs):
        self.name = name
        self.message = message
        self.items = items
        self.npc = npcs

#Locations and descriptions (most descriptions need to be changed, it's just a start)
l1 = Location('tavern', "You're in a small clearing with the tavern nearby.", [o2, o3], [])
l2 = Location('river', 'A large river, too ferocious to pass, rages on to the north of you. No living creature could survive currents like those.', [o6], [])
l3 = Location('forest', 'A dense forest stretches on to the south.', [], [])
l4 = Location('canyon', 'A canyon lays to the west of you, too steep to climb down and too wide to cross.', [], [])
l5 = Location('river/mountain', "To the north you hear river rapids and your east you see tall snowy mountains. Herbalist is here too- description?", [o7, o9], [])
l6 = Location('river/canyon', "To the north a river rages on and to the west you see a canyon. Human camp here!!", [], [])
l7 = Location('canyon/forest',"A canyon lays to the west and a great forest to the south. Big troll dude!!", [], [])
l8 = Location('forest/mountains', "You feel the chill of snowy mountains to the east and hear the sounds of a forest to the south. Antidote goes here but idk what it looks like.", [o8], [])
l9 = Location('mountains', "Tall mountains to the east tower above you. Elf territory! What's it look like", [], [])
l10 = Location('tavern main room', 'You enter the main room of the tavern. There is a door to your left.', [], [])
l11 = Location('tavern left room', "You walk through the door on the left and find yourself in a small inn.", [], [])
l12 = Location('human territory main', 'You enter a courtyard with humans wandering around.', [], [])
l13 = Location('human north', 'You see barracks.', [], [])
l14 = Location('human south', 'A large warehouse stands in front of you.', [], [])
l15 = Location('elf main', 'A tower stands tall in front of you.', [], [])
l16 = Location('elf north', 'You see a lush garden with all sorts of unidentifiable plants and fruits and vegetables.', [], [])
l17 = Location('elf south', 'You see a storage building guarded by elves.', [o4, o5], [])
l18 = Location('1', 'Forest', [], [])
l19 = Location('2', 'Forest', [], [])
l20 = Location('3', 'Forest', [], [])
l21 = Location('4', "You find yourself in a small clearing with dense forest all around.", [], [])
l22 = Location('5', 'Forest', [], [])
l23 = Location('6', 'Forest', [], [])
l24 = Location('7', 'Forest', [], [])
l25 = Location('8', 'Forest', [], [])
l26 = Location('9', 'Forest', [], [])
l27 = Location('10', 'Forest', [], [])
l28 = Location('11', 'Forest', [], [])
l29 = Location('12', 'Forest', [], [])
l30 = Location('13', 'Forest', [], [])
l31 = Location('14', 'Forest', [], [])
l32 = Location('15', 'Forest', [], [])
l33 = Location('16', 'Forest', [], [])
l34 = Location('17', 'Forest', [], [])
l35 = Location('18', 'Forest', [], [])
l36 = Location('19', 'Forest', [], [])
l37 = Location('20', 'Forest', [], [])
l38 = Location('21', 'Forest', [], [])
l39 = Location('22', 'Forest', [], [])
l40 = Location('23', 'Forest', [], [])
l41 = Location('24', 'Forest', [], [])
l42 = Location('25', 'Forest', [], [])
l43 = Location('26', 'Forest', [], [])
l44 = Location('27', 'Forest', [], [])
l45 = Location('28', 'Forest', [], [])
l46 = Location('29', 'Forest', [], [])
l47 = Location('30', 'Forest', [], [])
l48 = Location('31', 'Forest', [], [])
l49 = Location('32', 'Forest', [], [])
l50 = Location('33', 'Forest', [], [])
l51 = Location('34', 'Forest', [], [])
l52 = Location('35', 'Forest', [], [])
l53 = Location('36', 'Forest', [], [])
l54 = Location('37', "You enter a small clearing where a large beast looms over you. You see a small branch of the healing tree that looks like it might be easy to break off.", [o1], [])
l55 = Location('38', 'Forest', [], [])
l56 = Location('39', 'Forest', [], [])
l57 = Location('40', 'Forest', [], [])
l58 = Location('41', 'Forest', [], [])
l59 = Location('42', 'Forest', [], [])
l60 = Location('43', 'Forest', [], [])
l61 = Location('44', 'Forest', [], [])
l62 = Location('45', 'Forest', [], [])
l63 = Location('46', 'Forest', [], [])
l64 = Location('47', 'Forest', [], [])
l65 = Location('48', 'Forest', [], [])
l66 = Location('49', 'Forest', [], [])

def items(i, command, x, y):
    if 'take' in command:
        thing = command.lstrip('take ').lower()
        atLocation = False
        for item in locations[(x,y)].items:
            if item.name.lower() == thing:
                atLocation = True
                if item == o1:
                    if x == 28 and y == 25:
                        print("\nYou prepare to grab a branch of the healing tree.\n")
                        locations[(x,y)].items.remove(item)
                        return True
                    else:
                        print("You took the branch. It was added to your inventory.")
                        locations[(x,y)].items.remove(item)
                        i.add_item(item)
                        return False
                else:
                    i.add_item(item)
                    print(f"You took the {item.name.lower()}. It was added to your inventory.")
                locations[(x,y)].items.remove(item)
        if atLocation == False:
            print(f"I don't see a '{thing}' here.")
    elif 'drop' in command:
        thing = command.lstrip('drop ').lower()
        inInventory = False
        for item in i.items:
            if item.name.lower() == thing:
                inInventory = True
                i.remove_item(item)
                locations[(x,y)].items.append(item)
        if inInventory == False:
            print("I don't see this item in your inventory.")
    elif command == 'open inventory':
        i.show_items()
    return False

def beast(i):
    
    print()
    print("The beast won't be happy once it realizes what you're doing, so don't take too long in breaking it off. Maybe there's something in your inventory that can help speed up the process? To view your inventory enter 'open inventory'. To use something from it enter 'use __'.")
    print("\nUse commands 'go north', 'go south', 'go east', and 'go west' to get to the exit of the forest! Keep in mind there are many obstacles in the way such as logs, large trees, hedges, and more. The beast is chasing you, so try not to backtrack! If you get stuck and really need to, there might be something in your inventory that can help. You only have 20 moves to escape the forest before the beast catches you, but some inventory items will slow the beast down and give you extra moves. Good luck!")
    print("\nNow quick, take the branch!")
    print("\n")
    
    oginventory = []
    for item in i.items:
        oginventory.append(item)
    
    moves = 20
    print(moves, "moves left")
    moveslist = ['outta range']
    x = 28
    y = 25
    axed = False

    command = input().lower()
    
    while command:
        illegal = False
        
        if command == 'use axe':
            if o10 not in i.items:
                illegal = True
                print("There is no axe in your inventory!")
            elif o1 in i.items:
                illegal = True
                print("You throw the axe at the beast and it dodges it gracefully. Uh-oh...")
                i.remove_item(o10)
            else:
                illegal = True
                i.remove_item(o10)
                axed = True
                print("The axe saves you a lot of time and a branch falls to the ground. Take it, quick!")
        
        elif command == 'take branch':
            illegal = True
            if axed == True:
                i.add_item(o1)
                print("Uh-oh. The beast isn't too happy about you taking a branch off the healing tree. I hope you remember the path you took to get here- run!!")
            if axed == False:
                i.add_item(o1)
                print("If only you had used an axe or something. It takes a long time for you to break the branch off the tree and the beast is gaining on you quickly. I hope you remember the path you took to get here- run!!")
                moves -= 2
        
        elif command == 'go north':
            if y + 1 > 30:
                illegal = True
                print("The forest grows too dense to get through that direction. Look for a clearing to exit.") 
            elif (x,y+1) not in locations:
                illegal = True
                print("There is a large tree blocking the way! Try going another direction.")
            elif moveslist[-1] == 'go south':
                illegal = True
                moveslist.append('go north')
                print("You just came from that direction! If you go back now the beast will get you. Try going another direction or using something from your inventory to distract it while you run by. ")
            else:
                y += 1
        
        elif command == 'go east':
            if x + 1 > 33:
                illegal = True
                print("The forest grows too dense to get through that direction. Look for a clearing to exit.")
            elif (x+1,y) not in locations:
                illegal = True
                print("There is a fallen log blocking the way! Try going another direction.")
            elif moveslist[-1] == 'go west':
                illegal = True
                moveslist.append('go east')
                print("You just came from that direction! If you go back now the beast will get you. Try going another direction or using something from your inventory to distract it while you run by. ")
            else:
                x += 1
        
        elif command == 'go south':
            if y - 1 < 24:
                illegal = True
                print("The forest grows too dense to get through that direction. Look for a clearing to exit.")
            elif (x,y-1) not in locations:
                illegal = True
                print("There is a giant hedge blocking the way! Try going another direction.")
            elif moveslist[-1] == 'go north':
                illegal = True
                moveslist.append('go south')
                print("You just came from that direction! If you go back now the beast will get you. Try going another direction or using something from your inventory to distract it while you run by. ")
            else:
                y -= 1
        
        elif command == 'go west':
            if x - 1 < 27:
                illegal = True
                print("The forest grows too dense to get through that direction. Look for a clearing to exit.")
            elif (x-1,y) not in locations:
                illegal = True
                print("There is a boulder blocking the way! Try going another direction.")
            elif moveslist[-1] == 'go east':
                illegal = True
                moveslist.append('go west')
                print("You just came from that direction! If you go back now the beast will get you. Try going another direction or using something from your inventory to distract it while you run by. ")
            else:
                x -= 1
        
        elif command == 'use human food':
            if o2 not in i.items:
                illegal = True
                print("There's no human food in your inventory!")
            elif x == 28 and y == 25:
                illegal = True
                print("You're still in the clearing with the tree! Only use this valuable resource if you're stuck!")
            else:
                i.remove_item(o2)
                moves += 4
                print("You throw the food on the ground at the feet of the beast and it stops to eat- you've temporarily distracted it! You run past it in the direction you came from.")
                if moveslist[-1] == 'go north':
                    y+=1
                elif moveslist[-1] == 'go south':
                    y-=1
                elif moveslist[-1] == 'go east':
                    x+=1
                elif moveslist[-1] == 'go west':
                    x-=1
        
        elif command == 'use elven plants':
            if o3 not in i.items:
                illegal = True
                print("There are no elven plants in your inventory!")
            elif x == 28 and y == 25:
                illegal = True
                print("You're still in the clearing with the tree! Only use this valuable resource if you're stuck!")
            else:
                i.remove_item(o3)
                moves += 4
                print("You toss the magical plants behind you and they grow huge in an instant, blocking the path of the beast!")
                if moveslist[-1] == 'go north':
                    del locations[(x,y-1)]
                elif moveslist[-1] == 'go south':
                    del locations[(x,y+1)]
                elif moveslist[-1] == 'go east':
                    del locations[(x-1,y)]
                elif moveslist[-1] == 'go west':
                    del locations[(x+1,y)]
        
        elif command == 'open inventory':
            illegal = True
            print("\nInventory:")
            for item in i.items:
                print(item.name)
            print()
            moves += 1
        
        elif 'use ' in command:
            illegal = True
            invent = False
            thing = command.lstrip('use ').lower()
            for item in i.items:
                if item.name.lower() == thing:
                    invent = True
                    print("You have no use for this item against the beast.")
            if invent == False:
                print(f"I don't see a '{thing}' in your inventory.")

        if not illegal:
            if (x,y) != (28,25):
                print(locations[(x,y)].message)
            else:
                print("You're back at the clearing where you stole the healing tree from!")
            moveslist.append(command)
        
        moves-=1
        print("\n" + str(moves) + " moves left")
        
        if moves == 0:
            print("The beast catches you and throws you out of the forest. You collect yourself and look around. You're at the very edge of the forest to the south of the tavern.")
            return(False, oginventory)
        elif locations[(x,y)].message == "You find yourself in a small clearing with dense forest all around.":
            print("You escaped from the beast.")
            i.add_item(o1)
            return(True, oginventory)
        else:
            command = input().lower()



def motions(command, x, y):
    
    x = int(x)
    y = int(y)
    
    #edges of the main map
    max_coord_pos = 1.0
    max_coord_neg = -1.0

    illegal = False
    #checks to see if you're in the tavern
    if 15 > x > 5:
        intavern = True
    else:
        intavern = False
    #checks to see if you're anywhere but the main map and the tavern
    if x > 15:
        inside = True
    else:
        inside = False

    #base directional commands      
    if command == 'go north':
        if intavern == False: #if you're in the tavern it skips this cuz you can't use NESW
            if inside == False:
                if y + 1 > max_coord_pos:
                    illegal = True
                    print("The river is too fast to pass through.") 
                else:
                    y += 1.0
            else:
                if 25 > y+1 > 21:
                    illegal = True
                    print("A wall around the human territory prevents you from going that way.")
                elif 35 > y+1 > 30:
                    illegal = True
                    print("The forest is too thick to pass through here.")
                elif (x,y+1) not in locations:
                    illegal = True
                    print("There is a large tree blocking the way! Try going another direction.")
                elif y+1 > 41:
                    illegal = True
                    print("A wall around the elven territory prevents you from going that way.")
                else:
                    y+=1.0
        else:
            illegal = True
            print("Without seeing the sun, you have no sense of direction! You still know your lefts and rights though...")

    elif command == 'go east':
        if intavern == False:
            if inside == False:
                if x + 1 > max_coord_pos:
                    illegal = True
                    print("The mountains are too steep to climb.")
                else:
                    x += 1.0
            else:
                if 25 > x+1 > 20:
                    illegal = True
                    print("A wall around the human territory prevents you from going that way.")
                elif 35 > x+1 > 33:
                    illegal = True
                    print("The forest grows too dense to travel that direction.")
                elif (x+1,y) not in locations:
                    illegal = True
                    print("There is a fallen log blocking the way! Try going another direction.")
                elif x+1 > 40:
                    illegal = True
                    print("A wall around the elven territory prevents you from going that way.")
                else:
                    x+=1.0
        else:
            illegal = True
            print("Without seeing the sun, you have no sense of direction! You still know your lefts and rights though...")

    elif command == 'go south':
        if intavern == False:
            if inside == False:
                if y - 1 < max_coord_neg:
                    illegal = True
                    print("The forest is too overgrown to pass through.")
                else:
                    y -= 1.0
            else:
                if 39 > y-1 >35:
                    illegal = True
                    print("A wall around the elven territory prevents you from going that way.")
                elif 24 > y-1 > 20:
                    illegal = True
                    print("The forest grows too dense to travel that direction.")
                elif (x,y-1) not in locations:
                    illegal = True
                    print("There is a giant hedge blocking the way! Try going another direction.")
                elif 19 > y-1:
                    illegal = True
                    print("A wall around the human territory prevents you from going that way.")
                else:
                    y-=1.0
        else:
            illegal = True
            print("Without seeing the sun, you have no sense of direction! You still know your lefts and rights though...")

    elif command == 'go west':
        if intavern == False:
            if inside == False:
                if x - 1 < max_coord_neg:
                    illegal = True
                    print("The canyon is impassable.")
                else:
                    x -= 1.0
            else:
                if 40 > x-1 >35:
                    illegal = True
                    print("A wall around the elven territory prevents you from going that way.")
                elif 27 > x-1 > 25:
                    illegal = True
                    print("The forest grows too dense to travel that direction.")
                elif (x-1,y) not in locations:
                    illegal = True
                    print("There is a boulder blocking the way! Try going another direction.")
                elif 20 > x-1:
                    illegal = True
                    print("A wall around the human territory prevents you from going that way.")
                else:
                    x-=1
        else:
            illegal = True
            print("Without seeing the sun, you have no sense of direction! You still know your lefts and rights though...")

#tavern commands
    elif command == 'enter tavern':
        if intavern == True:
            illegal = True
            print("You're already in the tavern!")
        elif x != 0 or y != 0:
            illegal = True
            print("You don't see the tavern nearby.")
        else:
            y = 10
            x = 10
    elif command == 'go left':
        if intavern == True:
            if x - 1 < 9:
                illegal = True
                print("There is no door that way.")
            else:
                x-=1
        else:
            illegal = True
            print("It's easier to navigate using north, south, east, and west when you're outside.")
    elif command == 'go right':
        if intavern == True:
            if 18 > x + 1 > 10:
                illegal = True
                print("There is no door that way.")
            else:
                x+=1
        else:
            illegal = True
            print("It's easier to navigate using north, south, east, and west when you're outside.")
    elif command == 'exit tavern':
        if 15 < x or 8 > x or 15 < y or 8 > y:
            illegal = True
            print("You're not in the tavern.")
        elif y != 10 or x != 10:
            illegal = True
            print("The door to leave the tavern is not in this room.")
        else:
            x=0
            y=0

#human territory commands
    elif command == 'enter human territory':
        if inside == True:
            illegal = True
            print("You're already there!")
        elif x != -1 or y != 1:
            illegal = True
            print("You don't see an entrance nearby.")
        else:
            y = 20
            x = 20
    elif command == 'exit human territory':
        if 22 < x or 16 > x or 22 < y or 16 > y:
            illegal = True
            print("You're not in human territory.")
        else:
            x=-1
            y=1

#elf territory commands
    elif command == 'enter elf territory':
        if inside == True:
            illegal = True
            print("You're already there!")
        elif x != 1 or y != 0:
            illegal = True
            print("You don't see an entrance nearby.")
        else:
            y = 40
            x = 40
    elif command == 'exit elf territory':
        if x < 35 or y < 35:
            illegal = True
            print("You're not in elf territory.")
        else:
            x=1
            y=0

#forest commands
    elif command == 'enter forest':
        if inside == True:
            illegal = True
            print("You're already there!")
        elif x != 0 or y != -1:
            illegal = True
            print("You can't enter the forest from here.")
        else:
            y = 30
            x = 30
    elif command == 'exit forest':
        if 35 < x or 23 > x or 35 < y or 23 > y:
            illegal = True
            print("You're not in the forest.")
        elif y != 30 or x != 30:
            illegal = True
            print("The forest is too dense to exit from here.")
        else:
            x=0
            y=-1

    if not illegal:
        print(locations[(x,y)].message)

    return(x,y)
        

if __name__ == "__main__":
    
    print("You wake up in a tavern, unsure of where you are and what you're doing there. After talking to the tavernkeep you decide to explore. You exit the tavern and find yourself standing in a small clearing. Press 'i' to recieve further instruction.")
    #starting coords
    x = 0.0
    y = 0.0
    #dictionary of locations from above along with coordinates as keys
    locations = {
        (0.0,0.0) : l1, 
        (0.0,1.0) : l2, 
        (0.0,-1.0) : l3, 
        (-1.0,0.0) : l4, 
        (1.0,1.0) : l5, 
        (-1.0,1.0) : l6, 
        (-1.0, -1.0) : l7, 
        (1.0,-1.0) : l8,
        (1.0,0.0) : l9,
        (10,10) : l10,
        (9,10) : l11,
        (20,20) : l12,
        (20,21) : l13,
        (20,19) : l14,
        (40,40) : l15, 
        (40,41) : l16, 
        (40,39) : l17, 
        (27,30) : l18, 
        #(28,30) : l19, 
        (29,30) : l20, 
        (30,30) : l21, 
        (31,30) : l22, 
        (32,30) : l23, 
        #(33,30) : l24, 
        (27,29) : l25,
        (28,29) : l26,
        (29,29) : l27,
        (30,29) : l28,
        (31,29) : l29,
        (32,29) : l30,
        (33,29) : l31,
        #(27,28) : l32,
        (28,28) : l33,
        (29,28) : l34,
        #(30,28) : l35,
        (31,28) : l36,
        #(32,28) : l37,
        (33,28) : l38,
        (27,27) : l39,
        #(28,27) : l40,
        (29,27) : l41,
        (30,27) : l42,
        (31,27) : l43,
        (32,27) : l44,
        #(33,27) : l45,
        (27,26) : l46,
        (28,26) : l47,
        (29,26) : l48,
        (30,26) : l49,
        (31,26) : l50,
        #(32,26) : l51,
        (33,26) : l52,
        (27,25) : l53,
        (28,25) : l54,
        (29,25) : l55,
        #(30,25) : l56,
        (31,25) : l57,
        (32,25) : l58,
        (33,25) : l59,
        (27,24) : l60,
        (28,24) : l61,
        (29,24) : l62,
        (30,24) : l63,
        (31,24) : l64,
        #(32,24) : l65,
        (33,24) : l66,
        }
    #key words for motions
    motionlist = ['go', 'enter', 'exit']
    itemlist = ['take', 'drop', 'open']
    
    i = Inventory()
    command = input().lower()
    while command:
        if command == 'i':
            print("Welcome to the test map! Use commands 'go north', 'go east', 'go south', and 'go west' to navigate the world. To enter the tavern use the command 'enter tavern'. Once you're there, use the commands 'go left' and 'go right' to explore the building. When you wish to leave, enter 'exit tavern'. You can also enter the human and elf territories as well as the forest from certian locations. To do so, enter 'enter human/elf territory' or 'enter forest'. To return to the main map use 'exit human/elf territory' or 'exit forest'. You can always press 'i' to get a refresher on these instructions. Have fun!")
        else:
            commandlist = command.split()
            movementcommand = False
            for motion in motionlist:
                if motion in commandlist:
                    movementcommand = True
                    newcoords = motions(command, x, y)
                    x = newcoords[0]
                    y = newcoords[1]
            if movementcommand == False:
                for item in itemlist:
                    if item in commandlist:
                        beasttime = items(i, command, x, y)
                if beasttime == True:
                    escape = beast(i)
                    while escape:
                        if escape[0] == True:
                            x = 30
                            y = 30
                        else:
                            for item in escape[1]:
                                if item not in i.items:
                                    i.add_item(item)
                            i.remove_item(o1)
                            x = 0
                            y = -1
                
        command = input().lower()